# Pre-processamento para classificação de sentimentos

# Etapa 1: Importação e instalação das bibliotecas

In [1]:
import pandas as pd
import string
import spacy
import random
import seaborn as sns  
import numpy as np
import re
import os

# Etapa 2: Carregamento das bases de dados

- Fonte dos dados
: https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis#TweetsNeutralHash.csv

## Base de treinamento

- Negative label: 0
- Positive label: 1
- Neutro label: 2

In [22]:
df = pd.read_parquet('dataset/train.parquet')
print(df.shape)

(96521, 2)


In [12]:
df['sentimento'].value_counts()

sentimento
positivo    32634
negativo    32331
neutro      31556
Name: count, dtype: int64

# Etapa 3: Função para pré-processamento dos textos

- Letras minúsculas
- Nome do usuário (@)
- URLs
- Espaços em branco
- Emoticons
- Stop words
- Lematização
- Pontuações

In [ ]:
# !python -m spacy download pt_core_news_sm

In [13]:
pln = spacy.load('pt_core_news_sm')
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

In [14]:
df['texto'][1]

'Doria diz não ter preferência por oponente no segundo turno https://t.co/FXBktA1K2J'

In [15]:
print(stop_words)

{'seus', 'fazia', 'não', 'nuns', 'tivemos', 'todo', 'primeira', 'comprida', 'deste', 'toda', 'nas', 'então', 'foste', 'cá', 'esteve', 'usa', 'sois', 'quarta', 'devem', 'mas', 'zero', 'maior', 'muito', 'na', 'todas', 'valor', 'área', 'meu', 'vêm', 'segundo', 'sete', 'lado', 'mais', 'forma', 'adeus', 'umas', 'algumas', 'se', 'cento', 'outros', 'eu', 'vinda', 'ver', 'seu', 'ele', 'dezoito', 'você', 'dizem', 'quem', 'baixo', 'pegar', 'antes', 'sou', 'sempre', 'estar', 'dá', 'partir', 'no', 'certeza', 'debaixo', 'sétima', 'esta', 'tentei', 'sim', 'vão', 'desde', 'aquilo', 'vários', 'quarto', 'estará', 'estiveste', 'boa', 'lugar', 'vezes', 'porquanto', 'quê', 'obrigada', 'vai', 'ponto', 'tua', 'da', 'parece', 'nunca', 'duas', 'foram', 'apoio', 'poder', 'quinze', 'muitos', 'teus', 'mal', 'caminho', 'sétimo', 'tarde', 'três', 'fazemos', 'final', 'cima', 'vossas', 'és', 'às', 'catorze', 'novo', 'pelas', 'além', 'tempo', 'uns', 'conhecido', 'quer', 'desta', 'parte', 'em', 'vossa', 'do', 'esse', 

In [16]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
texto = "Nielsen Castelo Damasceno"
print(texto.lower())

nielsen castelo damasceno


In [31]:
import re

texto = "Olá @usuario123, tudo bem? Veja o perfil de @amigo_xyz para mais informações!"
texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", '', texto)
print(texto)


Olá  tudo bem? Veja o perfil de  para mais informações!


In [29]:
import re

texto = "Acesse meu site em https://exemplo.com e confira outros links como http://teste.org.br"
texto = re.sub(r"https?://[A-Za-z0-9./]+", '', texto)
print(texto)


Acesse meu site em  e confira outros links como 


In [32]:
import re

texto = "Este   é   um    texto  com     muitos espaços."
texto = re.sub(r" +", ' ', texto)
print(texto)


Este é um texto com muitos espaços.


In [33]:
texto = "Olá! Tudo bem? Aqui está um sorriso :d e outro sorriso :p."
texto = texto.replace(':d', '')
texto = texto.replace(':p', '')
print(texto)


Olá! Tudo bem? Aqui está um sorriso  e outro sorriso .


In [34]:
import re

texto = "O preço é 123 reais e o desconto é de 50%."
texto = re.sub(r'\d+', '', texto)
print(texto)


O preço é  reais e o desconto é de %.


In [35]:
import re

texto = "Hellooo, isso é muitooo legaaalll!!!"
texto = re.sub(r'(\w)\1(\1+)', r'\1', texto)
print(texto)


Hello, isso é muito legal!!!


In [18]:
def preprocessamento(texto):
  # Letras minúsculas
  texto = texto.lower()

  # Nome do usuário
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)

  # URLs
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)
  
  #Removendo todos os :d (sorriso) ou sorriso:p
  texto = texto.replace(':d','')
  texto = texto.replace(':p','')

  #Removendo os digitos
  texto = re.sub(r'\d+','',texto)

  #Removendo caracteres repetidos 3 ou mais vezes
  texto = re.sub(r'(\w)\1(\1+)',r'\1',texto)
  
  # Espaços em branco
  texto = re.sub(r" +", ' ', texto)
  
  #Removendo os espaços do inicio e final de cada frase
  texto = texto.strip()

  # Emoticons
  lista_emocoes = {':)': 'emocaopositiva',
                 ':d': 'emocaopositiva',
                 ':(': 'emocaonegativa',
                 ':|': 'emoçaoneutra',
                 ':/': 'emoçaoneutra'}

  for emocao in lista_emocoes:
    texto = texto.replace(emocao, lista_emocoes[emocao])

  # Lematização
  documento = pln(texto)

  lista = []
  for token in documento:
    lista.append(token.lemma_)

  # Stop words e pontuações
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

  return lista

In [20]:
texto_teste = '@behin_d_curtain :D Para :( mim, http://ncdd.com.br é precisamente o contrário :) Vem a chuva e vem a boa disposição :)'
resultado = preprocessamento(texto_teste)
resultado

'parir emocaonegativa mim precisamente o contrário emocaopositiva vir o chuva e vir o disposição emocaopositiva'

In [23]:
df['texto'] = df['texto'].astype(str)
df['texto_preprocess'] = df['texto'].apply(preprocessamento)

In [26]:

df.to_parquet('dataset/train_process.parquet')

In [27]:
df.head()

,texto,sentimento,texto_preprocess
0,Os minions no @BlogFarofeiros. Ainda bem que n...,positivo,o minions o responder emocaopositiva
1,Doria diz não ter preferência por oponente no ...,neutro,doria preferência oponente o segundar turno
2,@estervi50977812 @alicesaquino @RiverdaleBrasi...,positivo,e transmitir o temporada viver sincronia o w...
3,"@ourbookself Oi, Paloma :) Vamos ver o que aco...",positivo,oi palomar emocaopositiva o acontecer o e-mail...
4,Escolha de Jair Ventura mostra busca do Corint...,neutro,escolher jair venturo mostrar buscar corinthia...
